In [2]:
cd D:\Saarbrucken\EDA_Research\vae-disentanglement\disentanglement_lib_pl

D:\Saarbrucken\EDA_Research\vae-disentanglement\disentanglement_lib_pl


In [15]:
import torch
from torch import nn
import numpy as np
from architectures import encoders, decoders
from common.ops import Flatten3D, Unsqueeze3D, Reshape
from torch.nn import functional as F

In [3]:
enc = getattr(encoders, 'SimpleGaussianConv64CommAss')(10,3,64)
dec = getattr(decoders, 'SimpleConv64CommAss')(10,3,64)

In [4]:
img = torch.rand(size=[1,3,64,64])

In [11]:
enc = nn.Sequential(
    nn.Conv2d(1, 5, 2, 1, 0), # B,5,2,2
    nn.ReLU(True),
    Flatten3D(), # B,5x2x2
    nn.Linear(20, 10),
    nn.Linear(10, num_latent)
)

dec = nn.Sequential(
    Unsqueeze3D(),
    nn.Conv2d(num_latent, 20, 1, 1), 
    nn.ReLU(True),
    Reshape([5, 2, 2]),
    nn.ConvTranspose2d(5, 1, 2, 1, 0)
)



torch.Size([1, 256, 1, 1])


In [31]:
o = Unsqueeze3D()(o)
#print(o.shape, o)
o = nn.Conv2d(2, 20, 1, 1)(o)
o = nn.ReLU(True)(o)
#print(o.shape, o)
o = Reshape([5,2, 2])(o)
#print(o.shape, o)
o = nn.ConvTranspose2d(5, 1, 2, 1, 0)(o)
print(o.shape, o)

torch.Size([1, 1, 3, 3]) tensor([[[[-0.1105, -0.2154, -0.3674],
          [-0.1536, -0.5432, -0.3171],
          [-0.3573, -0.3726, -0.3949]]]],
       grad_fn=<SlowConvTranspose2DBackward>)


# Multiscale + GNN structure test 

In [1]:
cd D:\Saarbrucken\EDA_Research\vae-disentanglement\disentanglement_lib_pl

D:\Saarbrucken\EDA_Research\vae-disentanglement\disentanglement_lib_pl


In [53]:
import torch
import torch.nn as nn
from common.ops import Flatten3D, Unsqueeze3D, Reshape

class MultiScaleEncoder(nn.Module):
    """
    Encoder as used in 'Challenging Common Assumptions in the Unsupervised Learning of Disentangled Representations'
    """
    def __init__(self, feature_dim, in_channels, num_nodes):
        super().__init__()
        
        # Number of features per scale used by each node to compute initial node features
        # 3 means 3 features from each scale level will be used
        self.NUM_SCALES = 3
        self.feature_dim = feature_dim
        self.num_nodes = num_nodes
        self.features_to_take = self.feature_dim // self.num_nodes
        self.batch_size = None

        # in / out feature maps at each scale
        self.scale_3_in, self.scale_3_out = in_channels, 32
        self.scale_2_in, self.scale_2_out = 32, 32
        self.scale_1_in, self.scale_1_out = 32, 64

        # coarsest scale - outputs maps of shape B, 
        self.scale_3 = nn.Sequential(
            nn.Conv2d(self.scale_3_in, self.scale_3_out, 4, 2), # B, 32, 31 x 31
            nn.ReLU(True)
        )
        self.scale_3_feats = nn.Sequential(
            Flatten3D(),
            nn.Linear(self.scale_3_out * 31 * 31, self.feature_dim),
            nn.Tanh()
        )
        # mid scale - outputs maps of shape B, 
        self.scale_2 = nn.Sequential(
            nn.Conv2d(self.scale_2_in, self.scale_2_out, 4, 2), # B, 32, 14 x 14
            nn.ReLU(True)
        )
        self.scale_2_feats = nn.Sequential(
            Flatten3D(),
            nn.Linear(self.scale_2_out * 14 * 14, self.feature_dim),
            nn.Tanh()
        )
        
        # finest scale - outs maps of shape B,
        self.scale_1 = nn.Sequential(
            nn.Conv2d(self.scale_1_in, self.scale_1_out, 4, 2), # B, 64, 6 x 6
            nn.ReLU(True)
        )
        self.scale_1_feats = nn.Sequential(
            Flatten3D(),
            nn.Linear(self.scale_1_out * 6 * 6, self.feature_dim),
            nn.Tanh()
        )

    def forward(self, x):
        
        self.batch_size = x.shape[0]
        
        scale_3_x = self.scale_3(x)
        scale_3_feats = self.scale_3_feats(scale_3_x)

        scale_2_x = self.scale_2(scale_3_x)
        scale_2_feats = self.scale_2_feats(scale_2_x)

        scale_1_x = self.scale_1(scale_2_x)
        scale_1_feats = self.scale_1_feats(scale_1_x)
        #print(scale_3_feats.shape)
        #print(scale_2_feats.shape)
        #print(scale_1_feats.shape)
        
        # Just stacking gives the shape (3, batch_size, feature_dim). Hence, we need to permute tp get 
        # (batch_size, feature_dim, 3)
        multi_scale_feats = torch.stack([scale_3_feats, scale_2_feats, scale_1_feats]).permute(1,2,0)
        
        multi_scale_feats = multi_scale_feats.reshape(self.batch_size, self.num_nodes, self.NUM_SCALES * self.features_to_take )
        
        # reshape like this so that they can be associated with each latent node
        return multi_scale_feats

In [80]:
ms_enc = MultiScaleEncoder(4*2, 1, 4)

In [81]:
out = ms_enc(torch.randn(2,1,64,64))
print(out)
print(out.shape)

tensor([[[ 0.3083, -0.1329,  0.0334,  0.2024,  0.0683,  0.0298],
         [-0.1563,  0.0509,  0.0678,  0.1451, -0.1596,  0.0435],
         [-0.1692, -0.0438, -0.0202, -0.1336,  0.1160, -0.0684],
         [ 0.0177,  0.0900, -0.0045, -0.1646,  0.1573,  0.0504]],

        [[ 0.2070,  0.0479,  0.0158, -0.0309,  0.0621,  0.0077],
         [ 0.2464,  0.0283,  0.0510, -0.3034,  0.0586,  0.0516],
         [-0.1682,  0.0111, -0.0081,  0.2839, -0.0977, -0.0650],
         [-0.0549,  0.0296,  0.0189,  0.1485,  0.0469,  0.0219]]],
       grad_fn=<UnsafeViewBackward>)
torch.Size([2, 4, 6])


## Testing Prior GNN

In [48]:
class SimplePriorGNNLayer(nn.Module):
    """
    Implements GNN for P(Z|epsilon, A)
    """
    def __init__(self, in_node_feat_dim, out_node_feat_dim, adj_mat, is_final_layer=False):
        super().__init__()

        self.in_node_feat_dim = in_node_feat_dim
        self.out_node_feat_dim = out_node_feat_dim
        self.is_final_layer = is_final_layer
        self.A = adj_mat

        self.num_neighbours = self.A.sum(dim=-1, keepdims=True)
        self.projection = nn.Linear(self.in_node_feat_dim, self.out_node_feat_dim)
    
    def forward(self, node_feats):
        
        node_feats = self.projection(node_feats)
        node_feats = torch.matmul(self.A, node_feats)
        node_feats = node_feats / self.num_neighbours
        # I think this should only happen in the final layer?
        node_feats = torch.tanh(node_feats)
                
        if self.is_final_layer:
            # I think this split and reparam needs to happen only in the last layer
            
            # split into mu and sigma
            node_feats_mu, node_feats_logvar = node_feats.chunk(2, dim=2)
            
            # reparameterize
            #eps = torch.randn(size=(node_feats[1]))
            #node_feats_dist_params = node_feats_mu + eps * node_feats_logvar
            
            return node_feats_mu, node_feats_logvar
            
        return node_feats

In [49]:
# 4 nodes with topology given in A
dist_param_dim = 2
batch, V, node_feat_dim = 2, 4, 1
A = torch.Tensor([
    [1., 0., 1., 1.],
    [0., 1., 0., 1.],
    [0., 0., 1., 0.],
    [0., 0., 0., 1.]
])

# sample V exogenous vars per batch
E = torch.randn(size=(batch, V, node_feat_dim))

print(E)

tensor([[[ 1.2551e+00],
         [ 1.5454e+00],
         [-6.6605e-03],
         [ 1.7545e+00]],

        [[-2.2148e-01],
         [ 2.4578e-03],
         [-2.3859e-01],
         [-2.6712e+00]]])


In [50]:
# pass thru K GNN layers

prior_gnn = nn.Sequential(
    SimplePriorGNNLayer(in_node_feat_dim=1, out_node_feat_dim=2*dist_param_dim, adj_mat=A),
    SimplePriorGNNLayer(in_node_feat_dim=2*dist_param_dim, out_node_feat_dim=2*dist_param_dim, adj_mat=A, is_final_layer=True)
)


In [52]:
mus, logvars = prior_gnn(E)
print(mus)

print(logvars)

tensor([[[ 0.2724,  0.4389],
         [ 0.3510,  0.4798],
         [ 0.1645,  0.3794],
         [ 0.3544,  0.4811]],

        [[ 0.0453,  0.3144],
         [-0.0102,  0.2849],
         [ 0.1325,  0.3601],
         [-0.0374,  0.2759]]], grad_fn=<SplitBackward>)
tensor([[[ 0.4508, -0.4678],
         [ 0.4374, -0.5903],
         [ 0.4674, -0.2716],
         [ 0.4367, -0.5945]],

        [[ 0.4864, -0.0281],
         [ 0.4954,  0.0863],
         [ 0.4720, -0.2065],
         [ 0.5013,  0.1387]]], grad_fn=<SplitBackward>)


In [47]:
print(p.shape)
mus, logvars = p.chunk(2, dim=2)


print(mus.shape, mus)

print(logvars.shape, logvars)

torch.Size([2, 4, 4])
torch.Size([2, 4, 2]) tensor([[[0.0000, 0.3792],
         [0.0000, 0.4417],
         [0.0000, 0.4282],
         [0.0000, 0.3839]],

        [[0.0000, 0.2568],
         [0.0000, 0.1476],
         [0.0000, 0.3002],
         [0.0000, 0.1480]]], grad_fn=<SplitBackward>)
torch.Size([2, 4, 2]) tensor([[[0.5665, 0.0000],
         [0.5798, 0.0000],
         [0.5667, 0.0000],
         [0.5591, 0.0000]],

        [[0.5909, 0.0000],
         [0.6183, 0.0000],
         [0.5801, 0.0000],
         [0.6182, 0.0000]]], grad_fn=<SplitBackward>)


In [32]:
for p in prior_layer1.parameters(): print(p)

Parameter containing:
tensor([[-0.1122],
        [ 0.5648],
        [-0.3765],
        [ 0.2321]], requires_grad=True)
Parameter containing:
tensor([-0.9310,  0.2116,  0.6767,  0.8433], requires_grad=True)


## Testing Encoder GNN

In [ ]:
class SimpleGNNEncoderLayer(nn.Module):
    """
    Implements GNN for Q(Z|X,A)
    """
    def __init__(self, in_node_feat_dim, out_node_feat_dim, adj_mat, is_final_layer=False):
        super().__init__()

        self.in_node_feat_dim = in_node_feat_dim
        self.out_node_feat_dim = out_node_feat_dim
        self.A = adj_mat
        self.is_final_layer = is_final_layer

        self.num_neighbours = self.A.sum(dim=-1, keepdims=True)
        self.projection = nn.Linear(self.in_node_feat_dim, self.out_node_feat_dim)

    def forward(self, node_feats):
        # Num neighbours = number of incoming edges
        
        # for the first layer node-feats will be multi_scale_features from encoder
        
        node_feats = self.projection(node_feats)
        node_feats = torch.matmul(self.A, node_feats)
        node_feats = node_feats / self.num_neighbours
        node_feats = torch.tanh(node_feats)
        
        if self.is_final_layer:
            # I think this split and reparam needs to happen only in the last layer
            
            # split into mu and sigma
            node_feats_mu, node_feats_logvar = node_feats.chunk(2, dim=2)
            
            # reparameterize
            #eps = torch.randn(size=(node_feats[1]))
            #node_feats_dist_params = node_feats_mu + eps * node_feats_logvar
            
            return node_feats_mu, node_feats_logvar
        
        return node_feats